# Group Project 1 - Team Adolphe Quételet
### Team Members: Lara Gursoy(Project Manager), Jibin Qiu, Yingyin Li, Levi Ding, Qianhao Zhong

<br>

## **1. Project Goal:**

> __Our team is embarking on a comprehensive analysis of the global impact of the COVID-19 pandemic on the world's population. Our primary focus is centered on meticulously examining the mortality rates across various countries and diverse population groups, spanning the years from 2019 to 2022. With a profound commitment to uncovering the narratives concealed within the data, our analytical approach seeks to unveil the nuanced stories that emerge from this critical period of examination.__

<br>

## **2. Data Description:**

> **For this project, we are going to use [World Bank Group (wbdata)](https://wbdata.readthedocs.io/en/stable/), which is a simple Python interface to find and request information from the World Bank’s various databases. It can return data either as a dictionary containing full metadata or as a pandas DataFrame.**

> **Subgroups of Population and key variables:**
> * **Genders: Male, Female**
> * **Ages: 0-13, 14-17, 18-24, 25-44, 45-79, 80>=**
> * **Countries: World, US, China, Russia, India, French, UK**
> * **Years: 2019-2022**
> * **Mortality rate within different countries and the world**

<br>

## **3. Project Deliverables:**

### **[A] Data Import:**

In [1]:
%pip install wbdata
%pip install cufflinks

import wbdata
import pandas as pd
import cufflinks as cf
import numpy as np

  Using cached wbdata-0.3.0-py3-none-any.whl (14 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached cufflinks-0.17.3-py3-none-any.whl
  Using cached colorlover-0.3.0-py3-none-any.whl (8.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
wbdata.get_indicator(source= 40)

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [6]:
#wbdata.get_country()

### **[A] Population Statistics:**

In [7]:
def population(sex, year, age, country):
    """Function that takes in a SEX as a string ("Male", "Female"), a year, 
    a country as a string, an age range as integers (18, 24) and returns the MORTALITY RATE for the given given function arguments """
    if sex == "Male":
        labels = {"SP.DYN.AMRT" + ".MA": sex}
    elif sex == "Female":
        labels = {"SP.DYN.AMRT" + ".FE": sex}
    pop_stats = wbdata.get_dataframe(labels, country = country)
    pop_stats = pop_stats.filter(like = str(year), axis = 0)
    return pop_stats[sex][0] 

In [17]:
variable_labels = {"SP.POP.TOTL":"World Population", "SP.DYN.AMRT.FE":"Mortality rate adult female"}

world = wbdata.get_dataframe(variable_labels, country="WLD")

# Date index is of type string; change to integers
world.index = world.index.astype(int)

# Print a few years' data
world.head()

,World Population,Mortality rate adult female
date,,
2022,7.950947e+09,NaN
2021,7.888306e+09,138.473422
2020,7.820206e+09,119.449662
2019,7.741775e+09,111.119110
2018,7.660371e+09,112.376014


In [81]:
indicators = {"NY.GDP.PCAP.CD":"GDP per capita",
              "SP.DYN.TFRT.IN":"Total Fertility Rate",
              "SP.POP.GROW":"Population Growth Rate",
              "SP.DYN.AMRT.MA":"Male Mortality",
              "SP.DYN.AMRT.FE":"Female Mortality",
              "SP.POP.1564.FE.ZS":"% Adult Female",
              "SP.POP.TOTL.FE.ZS":"% Female"}

data = wbdata.get_dataframe(indicators, country=['USA', 'WLD'])


# Make years ints instead of strings
data.reset_index(inplace=True)


selected_years = ['2019', '2020', '2021', '2022']
data = data[data['date'].isin(selected_years)]
data.set_index(['country','date'],inplace=True)
data

GDP per capita  Total Fertility Rate  \
country       date                                         
United States 2022    76329.582265                   NaN   
              2021    70219.472454              1.664000   
              2020    63528.634303              1.641500   
              2019    65120.394663              1.706000   
World         2022    12743.851677                   NaN   
              2021    12363.830790              2.273163   
              2020    10902.237313              2.299666   
              2019    11338.150319              2.360609   

                    Population Growth Rate  Male Mortality  Female Mortality  \
country       date                                                             
United States 2022                0.377565             NaN               NaN   
              2021                0.156747             NaN               NaN   
              2020                0.964348      162.991000         92.267000   
              2019                0.455381      136.830000         79.455000   
World         2022                0.794101             NaN               NaN   
              2021                0.870822      206.353750        138.473422   
              2020                1.013088      182.409281        119.449662   
              2019                1.062657      170.291232        111.119110   

                    % Adult Female   % Female  
country       date                             
United States 2022       64.048400  50.486799  
              2021       64.245518  50.462072  
              2020       64.463633  50.436562  
              2019       64.694396  50.437008  
World         2022       64.376022  49.735282  
              2021       64.378919  49.722324  
              2020       64.400284  49.705180  
              2019       64.465123  49.691859

### **[A] Unit tests:**

In [10]:
population("Female", 2020, (18,24), "WLD")

119.449661608523

### **[A] Population DataFrames:**

### **[B] Population Pyramids:**

### **[C] Animated Population Pyramids:**

### **[C] Population Maps:**

### **[C] Other Visualization Tools:**

### **[C] Agricultural Supply:**

<br>

## **4. Presentation Link:**

<br>

## **5. Project Conclusion:**